# Práctica 2: Algoritmos de búsqueda informada

**Ingeniería Electrónica**

**Inteligencia Artificial**

**09/04/2021**

Los algoritmos de búsqueda informada utilizan conocimiento del dominio del problema para guiar la búsqueda hacia la solución. Esto se logra a través del uso de una función _heurística_.

## Funciones heurísticas

Una función heurística es una estimación del costo a la meta u objetivo desde cualquier estado del grafo de estados-acciones.

### El rompecabezas del 15

En el rompecabezas del 15 una posible función heurística consiste en contar el número de fichas desordenadas a partir de la configuración inicial.
Se incluye el código del rompecabezas antes visto.

In [1]:
from abc import ABC, abstractmethod

class EstadoProblema:
    """
    La clase EstadoProblema es abstracta.
    Representa un estado o configuración del problema a resolver.
    
    Es una interfaz simplificada para utilizarse
    en los algoritmos de búsqueda del curso.
    
    Al definir un problema particular hay que implementar los métodos
    abstractos
    """
    
    @abstractmethod
    def expand():
        """
        :return: el conjunto de estados sucesores
        """
        pass
    
    @abstractmethod
    def get_depth():
        """
        :return: la profundidad del estado
        """
        pass
        
    @abstractmethod
    def get_parent():
        """
        :return: referencia al estado predecesor o padre
        """
        pass

In [2]:
from functools import reduce
import random    

# La secuencia del 0 al 15
# 0 representará el espacio en blanco
seq = list(range(0,16))

# Cuatro posibles acciones para nuestro agente
# Mover una ficha en dirección: 
# izquierda (E), derecha (W), arriba (N), o abajo (S)
actions = ['E','W','N','S']

# Representaremos las configuraciones con bits
# Definimos algunas funciones útiles
# Recorre un bloque de 4 bits de unos a la posición i
x_mask = lambda i: 15<<(4*i)

# Extrae los cuatro bits que están en la posción i
# en la configuración c
# El rompecabezas tiene 16 posiciones (16X4 = 64 bits)
extract = lambda i,c: (c&(x_mask(i)))>>(4*i)

# Verifica si la posición z es la última columna
e_most = lambda z: (z%4)==3

# Verifica si la posición z es la primera columna
w_most = lambda z: (z%4)==0

# Verifica si la posición z es el primer renglón
n_most = lambda z: z<=3

# Verifica si la posición z es el último renglón
s_most = lambda z:z>=12

# Establecemos un diccionario con las acciones posibles
# para cada posición del rompecabezas
valid_moves = {i:list(filter(lambda action:\
((not action=='E') or (not e_most(i))) and \
((not action=='W') or (not w_most(i))) and \
((not action=='S') or (not s_most(i))) and \
((not action=='N') or (not n_most(i))),actions)) for i in seq}

# Realiza el movimiento hacía la izquierda
def move_east(puzzle):
    """
    :param puzzle: el rompecabezas
    """
    if(not e_most(puzzle.zero)):
        puzzle.zero += 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)>>4 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía la derecha
def move_west(puzzle):
    if(not w_most(puzzle.zero)):
        puzzle.zero -= 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)<<4 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía arriba
def move_north(puzzle):
    if(not n_most(puzzle.zero)):
        puzzle.zero -= 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)<<16 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía abajo
def move_south(puzzle):
    if(not s_most(puzzle.zero)):
        puzzle.zero += 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)>>16 | \
        (puzzle.configuration&~mask)

class Puzzle(EstadoProblema):
    """
    Rompecabezas del 15
    """
    
    
    def __init__(self, parent=None, action =None, depth=0):
        """
        Puede crearse un rompecabezas ordenado al no especificar
        parámetros del constructor.
        También se puede crear una nueva configuración a 
        partir de una configuración dada en parent.
        :param parent: configuración de referencia.
        :param action: la acción que se aplica a parent para
        generar la configuración sucesora.
        :depth la profundidad del estado a crear
        """
        self.parent = parent
        self.depth = depth
        if(parent == None):
            self.configuration =  \
                reduce(lambda x,y: x | (y << 4*(y-1)), seq)
            # posición del cero
            self.zero = 15
        else:
            self.configuration = parent.configuration
            self.zero = parent.zero
            if(action != None):
                self.move(action)

    def __str__(self):
        """
        :return: un string que representa 
        la configuración del rompecabezas
        """
        return '\n'+''.join(list(map(lambda i:\
        format(extract(i,self.configuration)," x")+\
        ('\n' if (i+1)%4==0 else ''),seq)))+'\n'

    def __repr__(self):
        """
        :return: representación texto de la configuración
        """
        return self.__str__()

    def __eq__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        tienen la misma configuración.
        """
        return (isinstance(other, self.__class__)) and \
        (self.configuration==other.configuration)

    def __ne__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        no tienen la misma configuración
        """
        return not self.__eq__(other)
        
    def __lt__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando la profundidad del objeto
        es menor que la del argumento
        """
        return self.depth < other.depth

    def __hash__(self):
        """
        :return: un número hash para poder usar la configuración en 
        un diccionario, delegamos al hash de un entero
        """
        return hash(self.configuration)

    def move(self,action):
        """
        Realiza un movimiento de ficha.
        Debemos imaginar que el espacio se mueve en la dirección
        especificada por acción
        :param action: la acción a realizar
        """
        if(action =='E'):
            move_east(self)
        if(action =='W'):
            move_west(self)
        if(action =='N'):
            move_north(self)
        if(action =='S'):
            move_south(self)
        return self


    @staticmethod
    def to_list(puzzle):
        """
        Convertimos la configuración a una lista de números
        :param puzzle: la configuración a convertir
        :return la lista con enteros
        """
        return [extract(i,puzzle.configuration) for i in seq]

    def shuffle(self,n):
        """
        Desordena de manera aleatoria el rompecabezas.
        :param n: el número de movimientos aleatorios a aplicar
        """
        for i in range(0,n):
            self.move(random.choice(valid_moves[self.zero]))
        return self

    def expand(self):
        """
        Los sucesores del estado, quitamos el estado padre
        """
        #filtering the path back to parent
        return list(filter(lambda x: \
        (x!=self.parent), \
        [Puzzle(self,action,self.depth+1) \
        for action in valid_moves[self.zero]]))
    
    def get_depth(self):
        """
        :return: la profundidad del estado
        """
        return self.depth
    
    def get_parent(self):
        """
        :return: el nodo predecesor (padre) del estado 
        """
        return self.parent

La clase __Puzzle__ incluye un método que hace simple el poder determinar el número de finchas fuera de lugar. 
El método  __to_list__.

In [3]:
# Ejemplo del método to_list
# Creamos un rompecabezas ordenado
ordenado = Puzzle()
print("rompecabezas ordenado:",ordenado)
lista_ordenada = Puzzle.to_list(ordenado)
print("el rompecabezas ordenado como una lista:\n",lista_ordenada)

rompecabezas ordenado: 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0


el rompecabezas ordenado como una lista:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]


In [4]:
# Si desordenamos el rompecabezas su lista ya no estará ordenada
from random import seed
seed(2019)
desordenado = Puzzle()
desordenado.shuffle(5)
print("rompecabezas desordenado:",desordenado)
lista_desordenada = Puzzle.to_list(desordenado)
print("el rompecabezas desordenado como una lista:\n",lista_desordenada)

rompecabezas desordenado: 
 1 2 3 4
 5 6 7 8
 9 0 a b
 d e f c


el rompecabezas desordenado como una lista:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 13, 14, 15, 12]


Se define la heurística __h1__ como el número de fichas fuera de lugar.

In [5]:
def h1(p_1,p_2):
    # cuenta el número de fichas que no están en orden
    return sum(1 \
    for a,b in zip(Puzzle.to_list(p_1),Puzzle.to_list(p_2)) if a!=b)

In [6]:
print("ordenado:",ordenado)
print("desordenado:",desordenado)
print("número de fichas fuera de lugar:",h1(ordenado,desordenado))

ordenado: 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0


desordenado: 
 1 2 3 4
 5 6 7 8
 9 0 a b
 d e f c


número de fichas fuera de lugar: 4


## Algoritmo A*

El algorimo A\* utiliza una función heurística para priorizar la expansión de los estados.
Primero el método para recuperar la ruta a partir de un estado.

In [7]:
from collections import deque

# trajectory nos regresará la trayectoria a partir de un estado
def trajectory(end):
    # nos valemos de un deque para almacenar la ruta
    sequence = deque()
    # agregamos el estado final o meta
    sequence.append(end)
    # nos vamos regresando al estado predecesor mientras este exista
    while end.get_parent():
        # nos movemos al predecesor
        end = end.get_parent()
        # lo agregamos a la lista
        sequence.append(end)
    # invertimos el orden de la secuencia
    sequence.reverse()
    # lo regresamos como una lista
    return list(sequence)

A continuación se muestra una implementación de A* basada en una cola de prioridad.

In [8]:
import heapq

class AStar:
    """
    Implementación del algoritmo A*
    """
    @staticmethod
    def search(origen,stop,g,h):
        """
        Búsqueda informada A*
        :param origen: estado inicial
        :param stop: función de paro, verdadera para el estado meta
        :param g: función de costo acumulado
        :param h: función heurística, costo estimado a la meta
        """
        # Nuestra cola de prioridad
        agenda = []
        # Conjunto de estados expandidos
        expandidos = set()
        # Condición trivial
        if stop(origen):
            return trajectory(origen)
        
        # Estado inicial a la cola de prioridad
        # La prioridad será f(s) = g(s) + h(s), 
        # para s una configuración
        f = lambda s: g(s) + h(s)
        
        # Agregamos el origen a la agenda
        heapq.heappush(agenda,(f(origen),origen))
        
        # Mientras la agenda no este vacía
        while agenda:
            # El frente de la cola de prioridad es la configuración
            # de menor costo f
            nodo = heapq.heappop(agenda)[1]
            # Agregamos el estado a la lista de expandidos
            expandidos.add(nodo)
            # En A* es necesario verificar la condición de 
            # paro tras sacar el elemento de la agenda
            if stop(nodo):
                return trajectory(nodo)
            # Realizamos la expansión del vértice
            for sucesor in nodo.expand():
                # Agregamos a la cola de prioridad siempre que no se haya
                # expandido previamente
                if sucesor not in expandidos:
                    heapq.heappush(agenda,(f(sucesor),sucesor))
        # No hay ruta al nodo meta
        # instrucción redundante
        return None

A continuación se prueba el algoritmo.

In [9]:
seed(2019)
p = Puzzle()
# 40 movimientos aleatorios
p.shuffle(40)
print("rompecabezas a resolver:",p)

rompecabezas a resolver: 
 5 1 2 4
 7 6 3 8
 a b e c
 9 0 d f




In [10]:
# llamar al al algoritmo A*
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzle(), # detenerse si esta ordenado
                    lambda s:s.get_depth(), # el costo acumulado es la profunidad
                    lambda s:h1(s,Puzzle())) # la heurística h1
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)

la ruta encontrada: [
 5 1 2 4
 7 6 3 8
 a b e c
 9 0 d f

, 
 5 1 2 4
 7 6 3 8
 a b e c
 9 d 0 f

, 
 5 1 2 4
 7 6 3 8
 a b 0 c
 9 d e f

, 
 5 1 2 4
 7 6 3 8
 a 0 b c
 9 d e f

, 
 5 1 2 4
 7 0 3 8
 a 6 b c
 9 d e f

, 
 5 1 2 4
 0 7 3 8
 a 6 b c
 9 d e f

, 
 0 1 2 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 0 2 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 2 0 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 7 0 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 0 7 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 a 0 b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 0 a b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 0 d e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d 0 e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e 0 f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0

]
longitud de la ruta: 16


Se observa que la solución esta a profundidad 16.

## Algoritmos de Costo Uniforme (UCS) y Voraz Primero el Mejor (GBFS)

¿Qué sucedería si el algoritmo no utilizara información del dominio del problema?

El algoritmo se denomina de costo uniforme o UCS.

En este caso bastaría proponer una función heurística __h_ucs__ igual a cero.

In [11]:
# Para obtener el algortimo UCS no
# usamos información del dominio del problema
# hacemos h cero para toda configuración
h_ucs = lambda s: 0

# Invocamos al algoritmo A*, se comportará como UCS
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzle(), # detenerse si esta ordenado
                    lambda s:s.get_depth(), # el costo acumulado es la profunidad
                    h_ucs) # la heurística es cero para toda configuración
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)

la ruta encontrada: [
 5 1 2 4
 7 6 3 8
 a b e c
 9 0 d f

, 
 5 1 2 4
 7 6 3 8
 a b e c
 9 d 0 f

, 
 5 1 2 4
 7 6 3 8
 a b 0 c
 9 d e f

, 
 5 1 2 4
 7 6 3 8
 a 0 b c
 9 d e f

, 
 5 1 2 4
 7 0 3 8
 a 6 b c
 9 d e f

, 
 5 1 2 4
 0 7 3 8
 a 6 b c
 9 d e f

, 
 0 1 2 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 0 2 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 2 0 4
 5 7 3 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 7 0 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 0 7 8
 a 6 b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 a 0 b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 0 a b c
 9 d e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 0 d e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d 0 e f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e 0 f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0

]
longitud de la ruta: 16


In [12]:
# Para obtener el algortimo GBFS no tomamos
# en cuenta el costo acumulado
# solo nos fijamos en la estimación a la meta
# para ello hacemos la función de costo igual a cero
g_gbfs = lambda s:0

# Invocamos al algoritmo A*, se comportará como GBFS
ruta = AStar.search(p, # rompecabezas desordenado
                    lambda s:s==Puzzle(), # detenerse si esta ordenado

                    g_gbfs, # el costo acumulado no se tomará en cuenta
                    lambda s:h1(s,Puzzle())) # la heurística h1
print("la ruta encontrada:",ruta)
print("longitud de la ruta:",len(ruta)-1)

la ruta encontrada: [
 5 1 2 4
 7 6 3 8
 a b e c
 9 0 d f

, 
 5 1 2 4
 7 6 3 8
 a 0 e c
 9 b d f

, 
 5 1 2 4
 7 6 3 8
 0 a e c
 9 b d f

, 
 5 1 2 4
 0 6 3 8
 7 a e c
 9 b d f

, 
 0 1 2 4
 5 6 3 8
 7 a e c
 9 b d f

, 
 1 0 2 4
 5 6 3 8
 7 a e c
 9 b d f

, 
 1 2 0 4
 5 6 3 8
 7 a e c
 9 b d f

, 
 1 2 3 4
 5 6 0 8
 7 a e c
 9 b d f

, 
 1 2 3 4
 5 6 e 8
 7 a 0 c
 9 b d f

, 
 1 2 3 4
 5 6 e 8
 7 a d c
 9 b 0 f

, 
 1 2 3 4
 5 6 e 8
 7 a d c
 9 0 b f

, 
 1 2 3 4
 5 6 e 8
 7 0 d c
 9 a b f

, 
 1 2 3 4
 5 6 e 8
 7 d 0 c
 9 a b f

, 
 1 2 3 4
 5 6 e 8
 7 d b c
 9 a 0 f

, 
 1 2 3 4
 5 6 e 8
 7 d b c
 9 0 a f

, 
 1 2 3 4
 5 6 e 8
 7 0 b c
 9 d a f

, 
 1 2 3 4
 5 6 e 8
 0 7 b c
 9 d a f

, 
 1 2 3 4
 5 6 e 8
 9 7 b c
 0 d a f

, 
 1 2 3 4
 5 6 e 8
 9 7 b c
 d 0 a f

, 
 1 2 3 4
 5 6 e 8
 9 7 b c
 d a 0 f

, 
 1 2 3 4
 5 6 e 8
 9 7 0 c
 d a b f

, 
 1 2 3 4
 5 6 0 8
 9 7 e c
 d a b f

, 
 1 2 3 4
 5 0 6 8
 9 7 e c
 d a b f

, 
 1 2 3 4
 5 7 6 8
 9 0 e c
 d a b f

, 
 1 2 3 4
 5 7 6 8


Se observa que GBFS no encuentra la solución óptima, nos regresa una solución de 46 pasos.

In [13]:

import timeit
from functools import partial

# función de paro
stop = lambda s:s==Puzzle()
# función de costo acumulado
g = lambda s:s.get_depth()
#función heurística

h = lambda s:h1(s,Puzzle())

# definimos una función para A*
def a_star(p,stop,g,h):
    return AStar.search(p,stop,g,h)

# función para UCS
def ucs(p,stop,g):
    return AStar.search(p,stop,g,lambda s:0)
    
# función para GBFS
def gbfs(p,stop,h):
    return AStar.search(p,stop,lambda s:0,h)

# Invocamos el algoritmo A*, tomamos el tiempo que toma una ejecución
t = timeit.timeit(partial(a_star,p=p,stop = stop,g=g,h=h),number=1)
print("A* tomó %.4f segundos en encontrar la solución"%t)
# Invocamos el algoritmo UCS, tomamos el tiempo que toma una ejecución
t = timeit.timeit(partial(ucs,p=p,stop = stop,g=g),number=1)
print("UCS tomó %.4f segundos en encontrar la solución"%t)
# Invocamos el algoritmo GBFS, tomamos el tiempo que toma una ejecucióN
t = timeit.timeit(partial(gbfs,p=p,stop = stop,h=h),number=1)
print("GBFS tomó %.4f segundos en encontrar la solución"%t)

A* tomó 0.0142 segundos en encontrar la solución
UCS tomó 4.8930 segundos en encontrar la solución
GBFS tomó 0.1206 segundos en encontrar la solución


## Distancia de Manhattan en el rompecabezas del 15

La distancia de Manhattan es una mejor estimación al nodo meta que el número de fichas fuera de lugar.
Se implementará la distancia de Manhattan.

In [14]:
# La secuencia del 0 al 15
# 0 representará el espacio en blanco
seq = list(range(0,16))
# el renglón
row = lambda i: i//4
# la columna
col = lambda i: i%4

class ManhattanDistance:
    """
    Implementación de la distancia de Manhattan para el rompecabezas del 15
    """
    def __init__(self,target = Puzzle()):
        """
        Crea el objeto para la meta establecida
        :param target: configuración meta
        """
        self.target =target
        self.locations =self._find_locations(target)
        self.distances = self._precompute_distances(self.locations)
        
    def _find_locations(self,puzzle):
        """
        Encuentra la posición de cada ficha
        :param puzzle: el rompecabezas
        :return: las posiciones
        """
        locations = [None]*16
        for i in enumerate(Puzzle.to_list(puzzle)):
            locations[i[1]] = i[0]
        return locations
        
    def _precompute_distances(self,locations):
        """
        Precalcula distancias por posición
        :param locations: ubicación de las fichas
        :return: las distancias
        """
        distances = [[0]*16 for i in seq]
        for i in seq:
            for j in seq:
                distances[i][j] = abs(row(j)-row(locations[i]))+ \
                abs(col(j)-col(locations[i]))
        return distances
       
    def distance_to_target(self,puzzle):
        """
        Calcula la distancia de Manhattan al objetivo
        :param puzzle: la configuración 
        :return: la distancia
        """
        # no consideramos la posición del cero
        return sum(map(lambda i:self.distances[i[0]+1][i[1]],\
        enumerate(self._find_locations(puzzle)[1:])))

Se compara el valor de las heurísticas para la configuración revuelta.

In [15]:
m = ManhattanDistance(target=Puzzle())

h2 = m.distance_to_target

print("La heurística h1 (número de piezas desordenadas) es:",h1(p,Puzzle()))
print("La heurística h2 (distancia de Manhattan) es:",h2(p))

La heurística h1 (número de piezas desordenadas) es: 12
La heurística h2 (distancia de Manhattan) es: 14


Ambas heurísticas no sobreestiman (la distancia real es 16).

_h2_ es mejor que _h1_ porque da un valor más alto.

Ahora vamos a comparar los tiempos de los algoritmos con la heurística h2.

In [16]:
# Invocamos el algoritmo A*, tomamos el tiempo que toma una ejecución
t = timeit.timeit(partial(a_star,p=p,stop = stop,g=g,h=h2),number=1)
print("A* tomó %.4f segundos en encontrar la solución"%t)
# Invocamos el algoritmo UCS, tomamos el tiempo que toma una ejecución
t = timeit.timeit(partial(ucs,p=p,stop = stop,g=g),number=1)
print("UCS tomó %.4f segundos en encontrar la solución"%t)
# Invocamos el algoritmo GBFS, tomamos el tiempo que toma una ejecucióN
t = timeit.timeit(partial(gbfs,p=p,stop = stop,h=h2),number=1)
print("GBFS tomó %.4f segundos en encontrar la solución"%t)

A* tomó 0.0046 segundos en encontrar la solución
UCS tomó 5.3218 segundos en encontrar la solución
GBFS tomó 0.0017 segundos en encontrar la solución


**¿Qué se observa? ¿Qué efecto tiene en general el diseño de una heurística?**

Se observa que la neuristica h2 es mejor por que mejora los tiempos al momento de encontrar la solucion.